In [2]:
data_dir = 'C:/PhotoQualityDataset/'

In [3]:
import os
os.sys.path.append('../')
os.sys.path.append('C:\\Users\\Wang\\Anaconda3\\envs\\py27\\lib\\site-packages')

In [4]:
import glob
from scipy.misc import imread
import numpy as np
from sklearn.utils import shuffle
import pandas as pd
import cv2
import math
import random
import pickle

In [5]:
# https://stackoverflow.com/questions/16702966/rotate-image-and-crop-out-black-borders
def rotatedRectWithMaxArea(w, h, angle):
    """
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle (maximal area) within the rotated rectangle.
    """
    if w <= 0 or h <= 0:
        return 0,0

    width_is_longer = w >= h
    side_long, side_short = (w,h) if width_is_longer else (h,w)

    # since the solutions for angle, -angle and 180-angle are all the same,
    # if suffices to look at the first quadrant and the absolute values of sin,cos:
    sin_a, cos_a = abs(math.sin(angle)), abs(math.cos(angle))
    if side_short <= 2.*sin_a*cos_a*side_long:
        # half constrained case: two crop corners touch the longer side,
        #   the other two corners are on the mid-line parallel to the longer line
        x = 0.5*side_short
        wr,hr = (x/sin_a,x/cos_a) if width_is_longer else (x/cos_a,x/sin_a)
    else:
        # fully constrained case: crop touches all 4 sides
        cos_2a = cos_a*cos_a - sin_a*sin_a
        wr,hr = (w*cos_a - h*sin_a)/cos_2a, (h*cos_a - w*sin_a)/cos_2a

    return wr,hr

def rotate_image(image, angle):
    """
    Rotates an OpenCV 2 / NumPy image about it's centre by the given angle
    (in degrees). The returned image will be large enough to hold the entire
    new image, with a black background
    """

    # Get the image size
    # No that's not an error - NumPy stores image matricies backwards
    image_size = (image.shape[1], image.shape[0])
    image_center = tuple(np.array(image_size) / 2)

    # Convert the OpenCV 3x2 rotation matrix to 3x3
    rot_mat = np.vstack(
        [cv2.getRotationMatrix2D(image_center, angle, 1.0), [0, 0, 1]]
    )

    rot_mat_notranslate = np.matrix(rot_mat[0:2, 0:2])

    # Shorthand for below calcs
    image_w2 = image_size[0] * 0.5
    image_h2 = image_size[1] * 0.5

    # Obtain the rotated coordinates of the image corners
    rotated_coords = [
        (np.array([-image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([-image_w2, -image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2, -image_h2]) * rot_mat_notranslate).A[0]
    ]

    # Find the size of the new image
    x_coords = [pt[0] for pt in rotated_coords]
    x_pos = [x for x in x_coords if x > 0]
    x_neg = [x for x in x_coords if x < 0]

    y_coords = [pt[1] for pt in rotated_coords]
    y_pos = [y for y in y_coords if y > 0]
    y_neg = [y for y in y_coords if y < 0]

    right_bound = max(x_pos)
    left_bound = min(x_neg)
    top_bound = max(y_pos)
    bot_bound = min(y_neg)

    new_w = int(abs(right_bound - left_bound))
    new_h = int(abs(top_bound - bot_bound))

    # We require a translation matrix to keep the image centred
    trans_mat = np.matrix([
        [1, 0, int(new_w * 0.5 - image_w2)],
        [0, 1, int(new_h * 0.5 - image_h2)],
        [0, 0, 1]
    ])

    # Compute the tranform for the combined rotation and translation
    affine_mat = (np.matrix(trans_mat) * np.matrix(rot_mat))[0:2, :]

    # Apply the transform
    result = cv2.warpAffine(
        image,
        affine_mat,
        (new_w, new_h),
        flags=cv2.INTER_LINEAR
    )

    return result

def crop_around_center(image, width, height):
    """
    Given a NumPy / OpenCV 2 image, crops it to the given width and height,
    around it's centre point
    """

    image_size = (image.shape[1], image.shape[0])
    image_center = (int(image_size[0] * 0.5), int(image_size[1] * 0.5))

    if(width > image_size[0]):
        width = image_size[0]

    if(height > image_size[1]):
        height = image_size[1]

    x1 = int(image_center[0] - width * 0.5)
    x2 = int(image_center[0] + width * 0.5)
    y1 = int(image_center[1] - height * 0.5)
    y2 = int(image_center[1] + height * 0.5)
    

    return image[y1:y2, x1:x2]

def largest_rotated_rect(w, h, angle):
    """
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle within the rotated rectangle.

    Original JS code by 'Andri' and Magnus Hoff from Stack Overflow

    Converted to Python by Aaron Snoswell
    """

    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    bb_w = w * math.cos(alpha) + h * math.sin(alpha)
    bb_h = w * math.sin(alpha) + h * math.cos(alpha)

    gamma = math.atan2(bb_w, bb_w) if (w < h) else math.atan2(bb_w, bb_w)

    delta = math.pi - alpha - gamma

    length = h if (w < h) else w

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    return (
        bb_w - 2 * x,
        bb_h - 2 * y
    )


def imrotate(image,angle):
    image_height, image_width = image.shape[0:2]
    image_rotated = rotate_image(image, angle)
    image_rotated_cropped = crop_around_center(
            image_rotated,
            *largest_rotated_rect(
                image_width,
                image_height,
                math.radians(angle)
            )
        )
    return image_rotated_cropped


# create new dataset by rotating the old one

## first, binary label, rotate 30 degree to one side

In [ ]:
path_fromTo = [data_dir+'/HighQuality/architecture/',data_dir+'/HighQuality/architecture/rotate20/']
file_list = glob.glob(path_fromTo[0]+'*.jpg')
if not os.path.isdir(path_fromTo[1]):
    os.mkdir(path_fromTo[1])
for i,f in enumerate(file_list):
    img = cv2.imread(f)
    rotated = imrotate(img,20)
    cv2.imwrite(path_fromTo[1]+f.split('\\')[1],rotated)
    
#     cv2.imshow("Origin",img)
#     cv2.imshow("Rotated",rotated)
#     cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
run runBinaryArch.py --task rotate20

## dataset with random rotation

In [7]:
path_fromTo = [data_dir+'/HighQuality/architecture/',data_dir+'/HighQuality/architecture/rotateRandom/']
file_list = glob.glob(path_fromTo[0]+'*.jpg')
if not os.path.isdir(path_fromTo[1]):
    os.mkdir(path_fromTo[1])
for i,f in enumerate(file_list):
    img = cv2.imread(f)
    ang = random.randrange(-20,20)
    if i %10==0:
        print(ang)
    rotated = imrotate(img,ang)
    cv2.imwrite(path_fromTo[1]+f.split('\\')[1],rotated)

8
11
-1
-20
-17
-7
17
4
-11
7
3
12
-2
18
-10
-6
-10
19
-9
-11
8
19
-5
-14
-7
-15
3
19
-17
9
6
14
-20
2
-18
-3
2
-10
-15
-7
12
18
-16
-1
-4
-17
-2
-9
-11
-3
-11
-15
-5
17
-17
-7
-10
3
-17
3


## regression with label as rotation angle

In [6]:
path_fromTo = [data_dir+'/HighQuality/architecture/',data_dir+'/HighQuality/architecture/rotateRegression/']
file_list = glob.glob(path_fromTo[0]+'*.jpg')
if not os.path.isdir(path_fromTo[1]):
    os.mkdir(path_fromTo[1])
label = {}
for i,f in enumerate(file_list):
    img = cv2.imread(f)
    picName = f.split('\\')[1]
    label[picName] = {'name':[],'label':[]}
    for k in range(5):
        ang = random.randrange(-20,20)
        label[picName]['name'] += [str(k)+picName]
        label[picName]['label'] += [ang]
        rotated = imrotate(img,ang)
        cv2.imwrite(path_fromTo[1]+str(k)+picName,rotated)
pickle_out = open(path_fromTo[1]+"dict.pickle","wb")
pickle.dump(label, pickle_out)
pickle_out.close()